<a href="https://colab.research.google.com/github/SwaksharDeb/Activity-recognition/blob/master/HAR_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import array
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from numpy import savetxt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import scipy.stats as stats
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

# Importing the dataset

In [70]:
dataset_user04 = pd.read_csv("final_field_user04_dataset.csv")
dataset_user07 = pd.read_csv("final_field_user07_dataset.csv")
dataset_user08 = pd.read_csv("final_field_user08_dataset.csv")
dataset_user18 = pd.read_csv("final_field_user18_dataset.csv")
dataset_user38 = pd.read_csv("final_field_user38_dataset.csv")
dataset_user51 = pd.read_csv("final_field_user51_dataset.csv")


# Data processing

In [71]:
dataset = pd.concat([dataset_user04, dataset_user07, dataset_user08, dataset_user18, dataset_user38, dataset_user51], ignore_index = True)

In [72]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1779768 entries, 0 to 1779767
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   x       float64
 1   y       float64
 2   z       float64
 3   act_id  int64  
dtypes: float64(3), int64(1)
memory usage: 54.3 MB


In [73]:
dataset['act_id'].value_counts()

5     1074348
4      406725
7      108697
3       97370
12      47544
1       22547
6       15586
2        2698
10       2225
9        2028
Name: act_id, dtype: int64

## Balancing the dataset

In [74]:
act_id_1 = dataset[dataset['act_id']==1].head(2028).copy()
act_id_2 = dataset[dataset['act_id']==2].head(2028).copy()
act_id_3 = dataset[dataset['act_id']==3].head(2028).copy()
act_id_4 = dataset[dataset['act_id']==4].head(2028).copy()
act_id_5 = dataset[dataset['act_id']==5].head(2028).copy()
act_id_6 = dataset[dataset['act_id']==6].head(2028).copy()
act_id_7 = dataset[dataset['act_id']==7].head(2028).copy()
act_id_8 = dataset[dataset['act_id']==8].head(2028).copy()
act_id_9 = dataset[dataset['act_id']==9].head(2028).copy()
act_id_10 = dataset[dataset['act_id']==10].head(2028).copy()
act_id_11 = dataset[dataset['act_id']==11].head(2028).copy()
act_id_12 = dataset[dataset['act_id']==12].head(2028).copy()

In [75]:
balanced_data = pd.DataFrame()
balanced_data = balanced_data.append([act_id_1, act_id_2, act_id_3, act_id_4, act_id_5, act_id_6, act_id_7, act_id_8, act_id_9, act_id_10, act_id_11, act_id_12])
balanced_data.reset_index(drop=True, inplace = True)
balanced_data.shape

(20280, 4)

In [76]:
balanced_data.head()

,x,y,z,act_id
0,-0.153,6.397,5.773,1
1,-0.153,6.397,5.773,1
2,-2.068,9.079,5.823,1
3,-4.980,7.891,0.036,1
4,-4.980,7.891,0.036,1


In [77]:
act_id_12.head()

,x,y,z,act_id
98016,-1.494,7.508,7.098,12
98017,-1.494,7.508,7.098,12
98018,-1.494,7.508,7.098,12
98019,-1.494,7.508,7.098,12
98020,-0.996,10.496,2.565,12


In [78]:
balanced_data['act_id'].value_counts()

12    2028
10    2028
9     2028
7     2028
6     2028
5     2028
4     2028
3     2028
2     2028
1     2028
Name: act_id, dtype: int64

## Seperating features and label

In [79]:

X = balanced_data.iloc[:, [0, 1, 2]].values
Y = balanced_data.iloc[:, 3].values

Y = Y - 1

In [80]:
X.shape, Y.shape

((20280, 3), (20280,))

## Feature scaling

In [81]:
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

In [82]:
X_scaled.shape[0]

20280

In [83]:
X_scaled

array([[-0.26063886, -0.36850608,  1.48722484],
       [-0.26063886, -0.36850608,  1.48722484],
       [-0.71680772,  0.57349967,  1.50193583],
       ...,
       [ 1.99304971, -2.69577307, -0.31075172],
       [ 1.98399779, -2.69577307, -0.31075172],
       [ 1.98399779, -2.6690794 , -0.31016328]])

## Test train split

## Making the frame

In [84]:
Fs = 20
frame_size = Fs*4 # 80
hop_size = 1 #Fs*2 # 40

def get_frames(data, frame_size, hop_size, data_label):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, data.shape[0] - frame_size, hop_size):
        x = data[i: i + frame_size, 0]
        y = data[i: i + frame_size, 1]
        z = data[i: i + frame_size, 2]

        # Retrieve the most often used label in this segment
        label = stats.mode(data_label[i: i + frame_size])[0][0]
        frames.append([x, y, z])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

X_train, y_train = get_frames(X_scaled, frame_size, hop_size, Y)

In [85]:
X_train.shape, y_train.shape

((20200, 80, 3), (20200,))

## Train-Test split

In [86]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 0, stratify = y_train)

In [87]:
X_train.shape, X_val.shape

((18180, 80, 3), (2020, 80, 3))

In [88]:
X_train[0].shape, X_val[0].shape

((80, 3), (80, 3))

In [90]:
X_train = X_train.reshape(-1, frame_size, 3, 1)
X_val = X_val.reshape(-1, frame_size, 3, 1)

In [91]:
X_train[0].shape, X_test[0].shape, X_val[0].shape

((80, 3, 1), (80, 3, 1), (80, 3, 1))

In [92]:
y_train.shape, y_test.shape, y_val.shape

((18180,), (1948,), (2020,))

In [93]:
y_train[0:10]

array([8, 8, 3, 3, 6, 1, 8, 8, 2, 9])

# CNN Model

In [94]:
model = Sequential()
model.add(Conv2D(16, (2, 2), activation = 'relu', padding = 'same', input_shape = X_train[0].shape))
model.add(Dropout(0.1))

model.add(Conv2D(32, (2, 2), activation='relu', padding= 'same'))
model.add(Dropout(0.2))

#model.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
#model.add(Dropout(0.2))

model.add(Flatten())

#model.add(Dense(128, activation = 'relu'))
#model.add(Dropout(0.2))

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(12, activation='softmax'))

In [95]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 80, 3, 16)         80        
_________________________________________________________________
dropout_12 (Dropout)         (None, 80, 3, 16)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 80, 3, 32)         2080      
_________________________________________________________________
dropout_13 (Dropout)         (None, 80, 3, 32)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 7680)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                491584    
_________________________________________________________________
dropout_14 (Dropout)         (None, 64)               

In [96]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [97]:
history = model.fit(X_train, y_train, epochs = 100, validation_data= (X_val, y_val), verbose=1)

Epoch 1/100
569/569 [==============================] - 11s 19ms/step - loss: 1.6009 - accuracy: 0.4023 - val_loss: 1.2416 - val_accuracy: 0.5158
Epoch 2/100
569/569 [==============================] - 11s 19ms/step - loss: 1.2736 - accuracy: 0.5070 - val_loss: 0.9814 - val_accuracy: 0.6139
Epoch 3/100
569/569 [==============================] - 11s 19ms/step - loss: 1.1098 - accuracy: 0.5586 - val_loss: 0.8460 - val_accuracy: 0.6772
Epoch 4/100
569/569 [==============================] - 11s 19ms/step - loss: 0.9967 - accuracy: 0.5990 - val_loss: 0.7169 - val_accuracy: 0.7366
Epoch 5/100
569/569 [==============================] - 11s 19ms/step - loss: 0.9173 - accuracy: 0.6198 - val_loss: 0.6335 - val_accuracy: 0.7658
Epoch 6/100
569/569 [==============================] - 11s 19ms/step - loss: 0.8598 - accuracy: 0.6492 - val_loss: 0.5735 - val_accuracy: 0.7792
Epoch 7/100
569/569 [==============================] - 11s 19ms/step - loss: 0.8101 - accuracy: 0.6595 - val_loss: 0.5275 - val_ac